In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.streaming import *
from pyspark.sql.types import *

spark = SparkSession.builder.master("local").appName("MergeDataframe").getOrCreate()
sc = spark.sparkContext

In [4]:
# sql adaptive query execution adaptive.coalescePartitions.enabled will makr paritions dynamic
sc.setLogLevel("Error")
spark.conf.set("spark.sql.shuffle.partitions",3)
spark.conf.get("spark.sql.shuffle.partitions")

"""
Expiry Date calculation
"""

filepath = "file:///C:/Users/venka/PycharmProjects/pythonProject/dataset/"

df1=spark.read.option('delimiter','|').csv(filepath + "DateInterview.csv",header=True,inferSchema=True)
#.option('delimiter','~|') can't be more than one character error

df1.show(truncate=0)
df1.printSchema()

+----------+------------+--------------+--------+----+
|RechargeId|Rechargedate|Remaining_days|validity|_c4 |
+----------+------------+--------------+--------+----+
|R201623   |20200511    |1             |online  |NULL|
|R201873   |20200119    |110           |online  |NULL|
|R201999   |20200105    |35            |online  |NULL|
|R201951   |20191105    |215           |online  |NULL|
+----------+------------+--------------+--------+----+

root
 |-- RechargeId: string (nullable = true)
 |-- Rechargedate: integer (nullable = true)
 |-- Remaining_days: integer (nullable = true)
 |-- validity: string (nullable = true)
 |-- _c4: string (nullable = true)



In [10]:
from pyspark.sql.functions import date_add,to_date,col,expr

df1 = df1.withColumn("date_s",to_date("Rechargedate","yyyyMMdd"))
df1.show()
df1.printSchema()

+----------+------------+--------------+--------+----+----------+
|RechargeId|Rechargedate|Remaining_days|validity| _c4|    date_s|
+----------+------------+--------------+--------+----+----------+
|   R201623|    20200511|             1|  online|NULL|2020-05-11|
|   R201873|    20200119|           110|  online|NULL|2020-01-19|
|   R201999|    20200105|            35|  online|NULL|2020-01-05|
|   R201951|    20191105|           215|  online|NULL|2019-11-05|
+----------+------------+--------------+--------+----+----------+

root
 |-- RechargeId: string (nullable = true)
 |-- Rechargedate: integer (nullable = true)
 |-- Remaining_days: integer (nullable = true)
 |-- validity: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- date_s: date (nullable = true)



In [16]:
df1.select("*",expr("date_add(date_s,Remaining_days)")).show()

+----------+------------+--------------+--------+----+----------+--------------------------------+
|RechargeId|Rechargedate|Remaining_days|validity| _c4|    date_s|date_add(date_s, Remaining_days)|
+----------+------------+--------------+--------+----+----------+--------------------------------+
|   R201623|    20200511|             1|  online|NULL|2020-05-11|                      2020-05-12|
|   R201873|    20200119|           110|  online|NULL|2020-01-19|                      2020-05-08|
|   R201999|    20200105|            35|  online|NULL|2020-01-05|                      2020-02-09|
|   R201951|    20191105|           215|  online|NULL|2019-11-05|                      2020-06-07|
+----------+------------+--------------+--------+----+----------+--------------------------------+

